(quickstart)=

# Quickstart 

In [ ]:
#%config InlineBackend.figure_format = "retina"

## Fundamental parameters estimation with pyABC

This is a simple example of how we can use a package like [pyABC](https://pyabc.readthedocs.io/) to generate an estimation of the fundamental parameters for an observed cluster. The user can combine **ASteCA** with any other package of their choosing.

We start by loading the observed cluster as a `pandas.DataFile()` object:

In [ ]:
import pandas as pd

# Load the observed cluster with pandas
cluster_df = pd.read_csv("../_static/cluster.csv")

Next, we import the ``asteca`` package and define a ``cluster()`` object:

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../../"))

import asteca

my_cluster = asteca.cluster(
    cluster_df=cluster_df,
    magnitude="Gmag",
    e_mag="e_Gmag",
    color="BP-RP",
    e_color='e_BP-RP',
)

Now we load the PARSEC isochrone file defining an `isochrone()` object:

In [ ]:
# Isochrones parameters
isochs = asteca.isochrones(
    isochs_path="../_static/parsec/",
    magnitude={"Gmag": 6390.21},
    color={"G_BPmag": 5182.58, "G_RPmag": 7825.08},
)

To instantiate a `synthetic` object we pass the `isochs` object we just created:

In [ ]:
# Synthetic clusters parameters
synthcl = asteca.synthetic(isochs)

Now we calibrate it with the `my_cluster` object and a dictionary with fixed parameters. We chose `alpha, beta Rv, DR` as fixed parameters: 

In [ ]:
fix_params = {"alpha": 0.09, "beta": 0.94, "Rv": 3.1, "DR": 0.}
synthcl.calibrate(my_cluster, fix_params)

Finally, we define a `likelihood()` object, which will be used to quantify how similar our observed cluster is to the generated synthetic clusters:

In [ ]:
# Instantiate the likelihood
likelihood = asteca.likelihood(my_cluster)

We are now ready to begin the fundamental parameters estimation process with pyABC. pyABC works by minimizing the distance between our data (the observed cluster) and synthetic data (the synthetic clusters). We will need to convenience functions to do this.

The first function required is `model()`. This takes a dictionary with the parameters that are *not* included in `fix_params` when our `synthcl` object was calibrated. This dictionary is used to generate a synthetic cluster via the `generate()` method. The returned variable is a dictionary simply because this is what pyABC expects; this is not a requirement of **ASteCA**.

In [ ]:
def model(fit_params):
    """Generate synthetic cluster"""
    synth_clust = synthcl.generate(fit_params)
    # pyABC expects a dictionary from this function
    synth_dict = {"data": synth_clust}
    return synth_dict

Since the `likelihood()` object by default returns a value that *increases* to indicate that the observed data and the synthetic data are more similar, we need to invert this value so that it *decreases* when the synthetic data approaches the observed data (becaue pyABC wants to *minimize* this value). We thus define a `distance()` function that inverts `lkl` using the `max_lkl` value which is the likelihood of the observed data evaluated on the observed data. I.e.: the largest value that the lieklihood can ever return.

Notice that this function recieves two arguments from pyABC but we only require one, hence the second argument is dismissed. The function makes use of the dictionary generated by the `model()` function, containing the synthetic cluster.

In [ ]:
max_lkl = likelihood.max_lkl
def distance(synth_dict, _):
    """
    The likelihood is maximized for better fitted models but this distance requires
    minimization. Hence we normalize and invert.
    """
    lkl = likelihood.get(synth_dict["data"])
    return 1 - lkl / max_lkl

Now we are ready to import pyABC and define the priors for the parameters that are being estimated (i.e: those that were not fixed).

First we use the `min_max()` method of the `synthcl` object to return the minimum and maximum values of the isochrones we loaded initaially (if you know thes values or want to use a different range, this step can be skipped).

Notice that the priors defined this way require using the minimum value and the desired *range*, not the maximum value.

In [ ]:
import pyabc

met_min, met_max, loga_min, loga_max = synthcl.min_max()

# Define a pyABC Distribution(). Uniform distributions are employed for all the parameters
# here but the user can of course change this as desired. See the pyABC docs for more
# information.
priors = pyabc.Distribution(
    {
        "met": pyabc.RV("uniform", met_min, met_max - met_min),
        "loga": pyabc.RV("uniform", loga_min, loga_max - loga_min),
        "dm": pyabc.RV("uniform", 7, 10 - 7),
        "Av": pyabc.RV("uniform", 0, 2 - 0)
    }
)

We create an [ABCSMC](https://pyabc.readthedocs.io/en/latest/api/pyabc.inference.html#pyabc.inference.ABCSMC) object with the `model` and `distance` functions, as well as the priors defined earlier. A population of 100 is usually enough. The tempfile defined below is required by pyABC.

In [ ]:
# Define pyABC parameters
pop_size = 100
abc = pyabc.ABCSMC(
    model,
    priors,
    distance,
    population_size=pop_size,
    #sampler=pyabc.sampler.SingleCoreSampler()
)

# This is a temporary file required by pyABC
import os
import tempfile  # Used by pyABC
db_path = "sqlite:///" + os.path.join(tempfile.gettempdir(), "pyABC.db")
abc.new(db_path)

Finally, we can run pyABC to perform Approximate Bayesian Inference on our parameters. We set a 

In [ ]:
import datetime as dt
history = abc.run(max_walltime=dt.timedelta(hours=0, minutes=1))

In [ ]:
# Extract last iteration and weights
df, w = history.get_distribution()


for k in df.keys():
    _median = pyabc.weighted_statistics.weighted_median(df[k].values, w)
    _std = pyabc.weighted_statistics.weighted_std(df[k].values, w)
    print(k, _median, _std)
print(f"ESS: {pyabc.weighted_statistics.effective_sample_size(w)}")
final_dist = pyabc.inference_util.eps_from_hist(history)
print(f"Dist: {final_dist}")

# Estimate masses and binarity fraction
model_fit = {
    k: pyabc.weighted_statistics.weighted_median(df[k].values, w) for k in df.keys()
}
my_cluster.clust_plot(synthcl, model_fit)

In [ ]:
pyabc.settings.set_figure_params("pyabc")
pyabc.visualization.plot_histogram_matrix(history)